**importing and connecting to bucket**

In [1]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  WORKER_COUNT  PREEMPTIBLE_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-46a7  GCE       4                                       RUNNING  us-central1-a


In [2]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [3]:
import pyspark
import sys
from pyspark.sql import Row
from pyspark.sql import SQLContext
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import numpy as np
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Jan 15 08:25 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [5]:
from pyspark.sql import *
import pyspark.sql.functions as F
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [6]:
spark

In [7]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [8]:
import inverted_index_gcp

In [9]:
from inverted_index_gcp import *

In [10]:
# incase of reupload
import importlib

importlib.reload(inverted_index_gcp)

<module 'inverted_index_gcp' from '/home/dataproc/inverted_index_gcp.py'>

In [11]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'abody' 
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name != 'graphframes.sh' and b.name.startswith('multi'):
        paths.append(full_path+b.name)

**loading wikipedia files and extract nessecery subjects**

In [12]:
parquetFile = spark.read.parquet(*paths)

In [13]:
doc_and_title = parquetFile.select("id","title").rdd

In [14]:
doc_text_text = parquetFile.select("id","text").rdd

In [15]:
doc_and_anchor = parquetFile.select("id","anchor_text").rdd

In [16]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [17]:
from inverted_index_gcp import InvertedIndex

In [18]:
def write_pkl(base_dir, name, dict):
  with open(Path(base_dir) / f'{name}.pkl', 'wb') as f:
    pickle.dump(dict, f)
    

def read_pkl(base_dir, name):
  with open(Path(base_dir) / f'{name}.pkl', 'rb') as f:
      return(pickle.load(f))

In [19]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def tokenize_and_len(text):
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  tokens = [token for token in tokens if token not in all_stopwords]
  return len(tokens)

In [20]:
def tokenize(text):
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  tokens = [token for token in tokens if token not in all_stopwords]
  return tokens

**creating dictionaries to be used later**

In [ ]:
# # create dict of docs and titles and write to bucket
# doc_and_title = doc_and_title.collectAsMap()

# # upload to bucket
# bucket = client.get_bucket(bucket_name)
# blob = bucket.blob('doc_title.pkl')
# blob.upload_from_string(pickle.dumps(doc_and_title))

In [ ]:
# doc_dict = doc_text_text.map(lambda x: (x[0],tokenize_and_len(x[1])))  # dict: {doc_id: len doc}
# doc_dict = doc_dict.collectAsMap()

# # upload to bucket
# bucket = client.get_bucket(bucket_name)
# blob = bucket.blob('len_docs_body.pkl')
# blob.upload_from_string(pickle.dumps(doc_dict))

In [ ]:
# corpus_len = {'corpus':parquetFile.count()}

In [ ]:
# # creating dict
# # pickle file : dict representing the corpus size

# # upload to bucket
# bucket = client.get_bucket(bucket_name)
# blob = bucket.blob('corpus_len.pkl')
# blob.upload_from_string(pickle.dumps(corpus_len))

In [35]:
def word_count(text, doc_id):
  ''' Count the frequency of each word in `text` (title) that is not included in 
  `all_stopwords` and return entries that will go into our posting lists. 
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs 
      for example: [("Anarchism", (12, 5)), ...]
  '''
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  tokens = [token for token in tokens if token not in all_stopwords]
  counter = Counter(tokens)
  ret = []
  for term in counter:
    ret.append((term,(doc_id,counter[term])))
  return ret


# def docs_len(text,id):
#   tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
#   tokens = [token for token in tokens if token not in all_stopwords]
#   return [(id, len(tokens))]


def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples 
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
  ret_sorted = sorted(unsorted_pl, key= lambda x: x[0])
  return ret_sorted


def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''

  data = postings.map(lambda post: (post[0], len(post[1])))
  return data


def calculate_tfidf(postings):
  ''' Takes a posting list RDD and calculate the tfidf for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, list of (docid,tfidf)) pair.
  '''
  
  client = storage.Client()
  bucket = client.get_bucket(bucket_name)

  blob1 = bucket.blob('len_docs_body.pkl')  #(doc_id : len_doc)
  pkl1 = blob1.download_as_string()
  doc_dict = pickle.loads(pkl1)

  blob2 = bucket.blob('corpus_len.pkl')
  pkl2 = blob2.download_as_string()
  corpus_len = pickle.loads(pkl2)

  blob3 = bucket.blob('body_term_idf.pkl')
  pkl3 = blob3.download_as_string()
  body_term_idf = pickle.loads(pkl3)
    
  def cacl_tf_idf(row): # row = (term, post_list)
    idf = body_term_idf[row[0]]
    tf_idf = map(lambda tup :(tup[0], (tup[1]/doc_dict[tup[0]])*idf),row[1])
    return (row[0], list(tf_idf))

  data = postings.map(lambda x: cacl_tf_idf(x))

  return data


def partition_postings_and_write(postings):
  ''' A function that partitions the posting lists into buckets, writes out 
  all posting lists in a bucket to disk, and returns the posting locations for 
  each bucket. Partitioning should be done through the use of `token2bucket` 
  above. Writing to disk should use the function  `write_a_posting_list`, a 
  static method implemented in inverted_index_colab.py under the InvertedIndex 
  class. 
  Parameters:
  -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and 
      offsets its posting list was written to. See `write_a_posting_list` for 
      more details.
  '''
  map_func = lambda term: (token2bucket_id(term[0]),term)
  my_rdd = postings.map(map_func)
  my_rdd = my_rdd.groupByKey()
  my_rdd = my_rdd.map(lambda bucket : InvertedIndex.write_a_posting_list(bucket,bucket_name))
  return my_rdd


def get_posting_iter(index):
    """
    This function returning the iterator working with posting list.
    
    Parameters:
    ----------
    index: inverted index    
    """
    words, pls = zip(*index.posting_lists_iter())
    return words,pls


def read_posting_list(inverted, w):
  with closing(MultiFileReader()) as reader:
    locs = inverted.posting_locs[w]
    b = reader.read(locs, inverted.df[w] * TUPLE_SIZE)
    posting_list = []
    for i in range(inverted.df[w]):
      doc_id = int.from_bytes(b[i*TUPLE_SIZE:i*TUPLE_SIZE+4], 'big')
      tf = int.from_bytes(b[i*TUPLE_SIZE+4:(i+1)*TUPLE_SIZE], 'big')
      posting_list.append((doc_id, tf))
    
    return posting_list

In [36]:
def calculate_term_idf(df, corpus_len):
    tmp = corpus_len / df
    lg = np.log10(tmp)
    return lg

# text

In [38]:
# word counts map
word_counts = doc_text_text.flatMap(lambda x: word_count(x[1], x[0]))
# sorting by doc_id
postings = word_counts.groupByKey().mapValues(reduce_word_counts)
# filtering postings 
postings_filtered = postings.filter(lambda x: len(x[1])>50)  #dict is: {term: [(dic_id, tf)..()..]}


In [ ]:
body_term_df = calculate_df(postings_filtered)

#read from bucket corpud len
client = storage.Client()
bucket = client.get_bucket(bucket_name)
blob = bucket.blob('corpus_len.pkl')
pkl = blob.download_as_string()
corpus_len = pickle.loads(pkl)

body_term_idf = body_term_df.map(lambda x: (x[0], calculate_term_idf(x[1], corpus_len['corpus']))).collectAsMap()

#write the idf dict to bucket
bucket = client.get_bucket(bucket_name)
blob = bucket.blob('body_term_idf.pkl')
blob.upload_from_string(pickle.dumps(body_term_idf))

In [ ]:
# posting list of tfidf
postings_tfidf = calculate_tfidf(postings_filtered)

**now calculate norm for each doc**

In [ ]:
docs_tfidf = postings_tfidf.flatMap(lambda x: x[1]).groupByKey().mapValues(list) #return: doc id, list of tfidf scores

In [ ]:
def clac_tfidf_norm(list_of_tfidf_scores):
    ret = 0 
    for score in list_of_tfidf_scores:
        ret += np.power(score,2)
    return np.sqrt(ret)

In [ ]:
# doc_normalization = docs_tfidf.map(lambda x:(x[0], clac_tfidf_norm(x[1]))).collectAsMap()

In [ ]:
# write_pkl('.', 'doc_norma', doc_normalization)


# # upload to bucket
# bucket = client.get_bucket(bucket_name)
# blob = bucket.blob('doc_norma.pkl')
# blob.upload_from_string(pickle.dumps(doc_normalization))

In [ ]:
pl_len = postings_filtered.map(lambda x : (x[0],len(x[1]))).collectAsMap()
# upload to bucket
bucket = client.get_bucket(bucket_name)
blob = bucket.blob('pl_len_dict.pkl')
blob.upload_from_string(pickle.dumps(pl_len))

In [ ]:
# partition posting lists and write out
_ = partition_postings_and_write(postings_filtered).collect()

In [ ]:
# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
# for blob in client.list_blobs(bucket_name, prefix='postings_gcp'):
for blob in client.list_blobs(bucket_name, prefix='body_index'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)

In [ ]:
client = storage.Client()
bucket = client.get_bucket(bucket_name)
blob = bucket.blob('pl_len_dict.pkl')
pkl = blob.download_as_string()
pl_len = pickle.loads(pkl)

In [ ]:
# Create inverted index instance
inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted.df = pl_len
# write the global stats out
inverted.write_index('.', 'text_index')
# upload to gs
index_src = "text_index.pkl"
# index_dst = f'gs://{bucket_name}/postings_gcp/{index_src}'
index_dst = f'gs://{bucket_name}/{index_src}'
!gsutil cp $index_src $index_dst

In [ ]:
# בודקת אם זה עובד

In [53]:
client = storage.Client()
bucket = client.get_bucket(bucket_name)
blob = bucket.blob('text_index.pkl')
pkl = blob.download_as_string()
text_index = pickle.loads(pkl)

In [ ]:
a = text_index.read_posting_list('morty', 'body_index')
len(a)

# titles

In [41]:
title_words_count = doc_and_title.flatMap(lambda x: word_count(x[1], x[0]))
title_pls = title_words_count.groupByKey().mapValues(reduce_word_counts)
# df_title = calculate_df(title_pls).collectAsMap()

In [ ]:
_ = partition_postings_and_write(title_pls).collect()

In [ ]:
title_super_pl = defaultdict(list)
# for blob in client.list_blobs(bucket_name, prefix='postings_gcp'):
for blob in client.list_blobs(bucket_name, prefix='title_index'):
    if not blob.name.endswith("pickle"):
        continue
    with blob.open("rb") as f:
        posting_locs = pickle.load(f)
        for k, v in posting_locs.items():
            title_super_pl[k].extend(v)

In [ ]:
titles_InvertedIndex = InvertedIndex()
titles_InvertedIndex.posting_locs = title_super_pl
titles_InvertedIndex.df = df_title
titles_InvertedIndex.write_index('.', 'titles')

index_src = "titles.pkl"
index_dst = f'gs://{bucket_name}/{index_src}'
!gsutil cp $index_src $index_dst

In [55]:
#####  test
client = storage.Client()
bucket = client.get_bucket(bucket_name)
blob = bucket.blob('titles.pkl')
pkl = blob.download_as_string()
titles_index = pickle.loads(pkl)

In [56]:
##### test
len(titles_index.read_posting_list('morty', 'title_index'))

23

In [ ]:
titles_len_dict = doc_and_title.map(lambda x: (x[0],tokenize_and_len(x[1])))  # dict: {doc_id: len title}
titles_len_dict = titles_len_dict.collectAsMap()


# upload to bucket
bucket = client.get_bucket(bucket_name)
blob = bucket.blob('titles_len_dict.pkl')
blob.upload_from_string(pickle.dumps(titles_len_dict))

In [ ]:
df_title_rdd = calculate_df(title_pls)

#read from bucket corpus len
client = storage.Client()
bucket = client.get_bucket(bucket_name)
blob = bucket.blob('corpus_len.pkl')
pkl = blob.download_as_string()
corpus_len = pickle.loads(pkl)

title_term_idf = df_title_rdd.map(lambda x: (x[0], calculate_term_idf(x[1], corpus_len['corpus']))).collectAsMap()

#write the idf dict to bucket
bucket = client.get_bucket(bucket_name)
blob = bucket.blob('title_term_idf.pkl')
blob.upload_from_string(pickle.dumps(title_term_idf))

# anchor text

In [48]:
anchor_text = doc_and_anchor.flatMap(lambda x:x[1]).reduceByKey(lambda i,j:i + ', ' + j)
anchor_text = anchor_text.map(lambda i: Row(anchor_text=i[1], id=i[0]))
anchor_text_words_count = anchor_text.flatMap(lambda x: word_count(x[0], x[1]))
anchor_pls = anchor_text_words_count.groupByKey().mapValues(reduce_word_counts)
# df_anchor = calculate_df(anchor_pls).collectAsMap()
# df_anchor_rdd = calculate_df(anchor_pls)

In [ ]:
anchor_len_dict = anchor_text.map(lambda x: (x[1],tokenize_and_len(x[0])))  # dict: {doc_id: len anchor}
anchor_len_dict = anchor_len_dict.collectAsMap()


# upload to bucket
bucket = client.get_bucket(bucket_name)
blob = bucket.blob('anchor_len_dict.pkl')
blob.upload_from_string(pickle.dumps(anchor_len_dict))

In [ ]:

#read from bucket corpud len
client = storage.Client()
bucket = client.get_bucket(bucket_name)
blob = bucket.blob('corpus_len.pkl')
pkl = blob.download_as_string()
corpus_len = pickle.loads(pkl)

anchor_term_idf = df_anchor_rdd.map(lambda x: (x[0], calculate_term_idf(x[1], corpus_len['corpus']))).collectAsMap()

#write the idf dict to bucket
bucket = client.get_bucket(bucket_name)
blob = bucket.blob('anchor_term_idf.pkl')
blob.upload_from_string(pickle.dumps(anchor_term_idf))

In [ ]:
_ = partition_postings_and_write(anchor_pls).collect()

In [ ]:
anchor_super_pl = defaultdict(list)
# for blob in client.list_blobs(bucket_name, prefix='postings_gcp'):
for blob in client.list_blobs(bucket_name, prefix='anchor_index'):
    if not blob.name.endswith("pickle"):
        continue
    with blob.open("rb") as f:
        posting_locs = pickle.load(f)
        for k, v in posting_locs.items():
            anchor_super_pl[k].extend(v)

In [ ]:
anchor_InvertedIndex = InvertedIndex()
anchor_InvertedIndex.posting_locs = anchor_super_pl
anchor_InvertedIndex.df = df_anchor
anchor_InvertedIndex.write_index('.', 'anchor_text')

index_src = "anchor_text.pkl"
index_dst = f'gs://{bucket_name}/{index_src}'
!gsutil cp $index_src $index_dst

# read from bucket

In [24]:
client = storage.Client()
bucket = client.get_bucket(bucket_name)
blob = bucket.blob('len_docs_body.pkl')
pkl = blob.download_as_string()
body_docs_len = pickle.loads(pkl)

In [25]:
client = storage.Client()
bucket = client.get_bucket(bucket_name)
blob = bucket.blob('pl_len_dict.pkl')
pkl = blob.download_as_string()
pl_len_dict = pickle.loads(pkl)

In [26]:
client = storage.Client()
bucket = client.get_bucket(bucket_name)
blob = bucket.blob('titles_len_dict.pkl')
pkl = blob.download_as_string()
titles_len_dict = pickle.loads(pkl)

In [27]:
client = storage.Client()
bucket = client.get_bucket(bucket_name)
blob = bucket.blob('anchor_len_dict.pkl')
pkl = blob.download_as_string()
anchor_len_dict = pickle.loads(pkl)

In [28]:
client = storage.Client()
bucket = client.get_bucket(bucket_name)
blob = bucket.blob('corpus_len.pkl')
pkl = blob.download_as_string()
corpus_len = pickle.loads(pkl)

In [29]:
client = storage.Client()
bucket = client.get_bucket(bucket_name)
blob = bucket.blob('doc_norma.pkl')
pkl = blob.download_as_string()
doc_norma = pickle.loads(pkl)

In [30]:
client = storage.Client()
bucket = client.get_bucket(bucket_name)
blob = bucket.blob('body_term_idf.pkl')
pkl = blob.download_as_string()
body_term_idf = pickle.loads(pkl)

# for BM25

In [31]:
def calculate_term_idf_bm25(df, corpus_len):
    numerator = corpus_len-df+0.5
    denominator = df + 0.5
    ret = np.log10((numerator/denominator)+1)
    return ret

### body

In [32]:
body_avgdl = np.average(list(body_docs_len.values()))

In [39]:
body_term_df = calculate_df(postings_filtered)

In [40]:
body_term_idf_bm25 = body_term_df.map(lambda x: (x[0], calculate_term_idf_bm25(x[1], corpus_len['corpus']))).collectAsMap()

#write the idf dict to bucket
bucket = client.get_bucket(bucket_name)
blob = bucket.blob('body_idf_bm25.pkl')
blob.upload_from_string(pickle.dumps(body_term_idf_bm25))

### title

In [42]:
title_avgdl = np.average(list(titles_len_dict.values()))

In [43]:
title_term_df = calculate_df(title_pls)

In [44]:
title_term_idf_bm25 = title_term_df.map(lambda x: (x[0], calculate_term_idf_bm25(x[1], corpus_len['corpus']))).collectAsMap()

#write the idf dict to bucket
bucket = client.get_bucket(bucket_name)
blob = bucket.blob('title_idf_bm25.pkl')
blob.upload_from_string(pickle.dumps(title_term_idf_bm25))

### anchor text

In [45]:
anchor_avgdl = np.average(list(anchor_len_dict.values()))

In [49]:
anchor_term_df = calculate_df(anchor_pls)

In [50]:
anchor_term_idf_bm25 = anchor_term_df.map(lambda x: (x[0], calculate_term_idf_bm25(x[1], corpus_len['corpus']))).collectAsMap()

#write the idf dict to bucket
bucket = client.get_bucket(bucket_name)
blob = bucket.blob('anchor_idf_bm25.pkl')
blob.upload_from_string(pickle.dumps(anchor_term_idf_bm25))

In [51]:
# creating dict for avgdl
avgdl_dict = {'body': body_avgdl, 'title': title_avgdl, 'anchor_text': anchor_avgdl}

# writing to bucket
bucket = client.get_bucket(bucket_name)
blob = bucket.blob('avgdl_dict.pkl')
blob.upload_from_string(pickle.dumps(avgdl_dict))

# PageRank

In [52]:
def generate_graph(pages):
  ''' Compute the directed graph generated by wiki links.
  Parameters:
  -----------
    pages: RDD
      An RDD where each row consists of one wikipedia articles with 'id' and 
      'anchor_text'.
  Returns:
  --------
    edges: RDD
      An RDD where each row represents an edge in the directed graph created by
      the wikipedia links. The first entry should the source page id and the 
      second entry is the destination page id. No duplicates should be present. 
    vertices: RDD
      An RDD where each row represents a vetrix (node) in the directed graph 
      created by the wikipedia links. No duplicates should be present. 
  '''

  edges =  pages.flatMapValues(lambda page: page)
  get_tuples = lambda page: (page[0],page[1][0],)
  edges = edges.map(get_tuples).distinct()
   
  v1 = pages.flatMap(lambda x: x[1]) # get all anchor text
  v1 = v1.map(lambda page: (page[0],)) # get all target pages
  v2 = pages.map(lambda x: (x[0],)) # get all source pages
  vertices = v1.union(v2).distinct()

  return edges, vertices

In [ ]:
# construct the graph 
edges, vertices = generate_graph(doc_and_anchor)
# compute PageRank
edgesDF = edges.toDF(['src', 'dst']).repartition(124, 'src')
verticesDF = vertices.toDF(['id']).repartition(124, 'id')
g = GraphFrame(verticesDF, edgesDF)
pr_results = g.pageRank(resetProbability=0.15, maxIter=6)
pr = pr_results.vertices.select("id", "pagerank")
pr = pr.sort(col('pagerank').desc())
page_rank_dict = pr.rdd.collectAsMap()


In [ ]:
#write pade rank dict to bucket
bucket = client.get_bucket(bucket_name)
blob = bucket.blob('page_rank_dict.pkl')
blob.upload_from_string(pickle.dumps(page_rank_dict))

In [ ]:
# Paths
# Using user page views (as opposed to spiders and automated traffic) for the 
# month of August 2021
pv_path = 'https://dumps.wikimedia.org/other/pageview_complete/monthly/2021/2021-08/pageviews-202108-user.bz2'
p = Path(pv_path) 
pv_name = p.name
pv_temp = f'{p.stem}-4dedup.txt'
pv_clean = f'{p.stem}.pkl'
# Download the file (2.3GB) 
!wget -N $pv_path
# Filter for English pages, and keep just two fields: article ID (3) and monthly 
# total number of page views (5). Then, remove lines with article id or page 
# view values that are not a sequence of digits.
!bzcat $pv_name | grep "^en\.wikipedia" | cut -d' ' -f3,5 | grep -P "^\d+\s\d+$" > $pv_temp
# Create a Counter (dictionary) that sums up the pages views for the same 
# article, resulting in a mapping from article id to total page views.
wid2pv = Counter()
with open(pv_temp, 'rt') as f:
  for line in f:
    parts = line.split(' ')
    wid2pv.update({int(parts[0]): int(parts[1])})



In [ ]:
#write the page view dict to bucket
bucket = client.get_bucket(bucket_name)
blob = bucket.blob('page_view_dict.pkl')
blob.upload_from_string(pickle.dumps(wid2pv))